In [1]:
# generate surface file and subcortical file
# the input roivalue is the 392 vector saved in a txt file, e.g. roivalue.txt

import numpy as np
import os 
import nibabel as nib
import nibabel.processing


import os

In [16]:
os.chdir('/Users/emilyolafson/GIT/workbench')

atlas_dir = '/Users/emilyolafson/GIT/atlas_files/'
image_dir = '/Users/emilyolafson/GIT/ENIGMA/testfig/'
txt_dir = '/Users/emilyolagson/GIT/ENIGMA/testfig/'


In [24]:
atlas = nib.load(atlas_dir + 'fs86_dil1_allsubj_mode.nii.gz').get_fdata()
nodes = np.unique(atlas)
nodes = np.delete(nodes,0)
data = np.zeros(atlas.shape, dtype=np.float32)
scalar = np.genfromtxt(txt_dir + 'avgHaufe_fs86_regionalSDC_correlation_chronic copy.txt', dtype = "float32", delimiter = ',', usecols = 0)


In [25]:
for i,n in enumerate(nodes):
    data[atlas == n] = scalar[i]
    
sample_img = nib.load(atlas_dir + 'fs86_dil1_allsubj_mode.nii.gz')
save_file = image_dir + '_metrics.nii.gz' 

In [26]:
save_img = nib.Nifti1Image(data, sample_img.affine, sample_img.header)
save_img.set_data_dtype(data.dtype)
nib.save(save_img, save_file) 


In [30]:
filename = save_file
surf_prefix = filename.strip("fs86_dil1_allsubj_mode.nii.gz")
for hemi in ['L', 'R']: 
    os.chdir('/Users/emilyolafson/GIT/workbench/bin_macosx64')
    cmd = './wb_command -volume-to-surface-mapping '+  filename + ' /Users/emilyolafson/GIT/ENIGMA/testfig/HCP_S1200_GroupAvg_v1/S1200.' + hemi + '.midthickness_MSMAll.32k_fs_LR.surf.gii '+ surf_prefix + hemi + '.shape.gii -enclosing'
    os.system(cmd)
    cmd = './wb_command -metric-dilate ' + surf_prefix + hemi + '.shape.gii' + ' /Users/emilyolafson/GIT/ENIGMA/testfig/HCP_S1200_GroupAvg_v1/S1200.' + hemi + '.midthickness_MSMAll.32k_fs_LR.surf.gii 20 ' + surf_prefix + hemi + '_filled.shape.gii -nearest'
    os.system(cmd)

#os.remove(save_file)


In [31]:
cmd

'./wb_command -metric-dilate /Users/emilyolafson/GIT/ENIGMA/testfig/_metricR.shape.gii /Users/emilyolafson/GIT/ENIGMA/testfig/HCP_S1200_GroupAvg_v1/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii 20 /Users/emilyolafson/GIT/ENIGMA/testfig/_metricR_filled.shape.gii -nearest'

In [14]:
os.chdir('/Users/emilyolafson/GIT/workbench')

atlas_dir = '/Users/emilyolafson/GIT/atlas_files/'
image_dir = '/Users/emilyolafson/GIT/ENIGMA/testfig/'
txt_dir = '/Users/emilyolafson/GIT/ENIGMA/testfig/'

roivol = nib.load(atlas_dir + "fs86_dil1_allsubj_mode.nii.gz")
Vroi = roivol.get_fdata()
Vnew = np.zeros(Vroi.shape)
roidata = np.genfromtxt(txt_dir + 'avgHaufe_fs86_regionalSDC_correlation_chronic copy.txt', dtype = "float32", delimiter = ',', usecols = 0)

for i,v in enumerate(np.unique(Vroi[Vroi>0])):
    Vnew[Vroi == v] = roidata[i]
imgnew = nib.Nifti1Image(Vnew, affine = roivol.affine, header = roivol.header)
nib.save(imgnew, "newdata.nii.gz")


newdata = nib.load("newdata.nii.gz")
cc400 = nib.load(atlas_dir + "fs86_dil1_allsubj_mode.nii.gz")
atlas2 = nib.load(atlas_dir + "fs86_dil1_allsubj_mode_subcort.nii.gz")
atlas1=nibabel.processing.resample_from_to(atlas2, cc400, order=0)
V400 = cc400.get_fdata()
V1 = atlas1.get_fdata()
subcortvals = np.unique(V400[(V400>0) * (V1>0)])
V400_subcort = V400 * np.isin(V400,subcortvals)
cc400_subcort = nib.Nifti1Image(V400_subcort, affine = cc400.affine, header = cc400.header)
#nib.save(cc400_subcort,"cc400_new1mm_subcort.nii.gz")
Vnew = newdata.get_fdata()*(V400_subcort>0)
newdata_subcort=nib.Nifti1Image(Vnew, affine=cc400.affine, header=cc400.header)
nib.save(newdata_subcort, image_dir + "_subcort.nii.gz")
os.remove("newdata.nii.gz")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

scalar = np.genfromtxt('avgHaufe_fs86_regionalSDC_correlation_chronic copy.txt', dtype = "float32", delimiter = ',', usecols = 0)
atlas = nib.load('cc400_roi_atlas.nii.gz').get_fdata()
nodes = np.unique(atlas)
nodes = np.delete(nodes,0)
data = np.zeros(atlas.shape)

for i,n in enumerate(nodes):
    data[atlas == n] = scalar[i]
sample_img = nib.load('fs86_dil1_allsubj_mode.nii.gz')
save_file = 'test_metrics.nii.gz' ## Name of file you want to save the metrics in 
save_img = nib.Nifti1Image(data, sample_img.affine, sample_img.header)
nib.save(save_img, save_file)  
curr_dir = os.getcwd()
save_file = curr_dir + '/cc400_roi_atlas_metrics.nii.gz'
surf_prefix = save_file.strip(".nii.gz")
for hemi in ['L', 'R']: 
    os.chdir('/home/yournetid/workbench/bin_linux64')
    cmd = './wb_command -volume-to-surface-mapping '+  save_file + ' /home/yournetid/workbench/HCP_S1200_GroupAvg_v1/S1200.' + hemi + '.midthickness_MSMAll.32k_fs_LR.surf.gii '+ surf_prefix + hemi + '.shape.gii -trilinear ;'
    os.system(cmd)
os.chdir(curr_dir)


In [ ]:
for hemi in ['L', 'R']: 
    os.chdir('/Users/emilyolafson/GIT/workbench/bin_macosx64')
    cmd = './wb_command -volume-to-surface-mapping '+  filename + '/Users/emilyolafson/GIT/ENIGMA/testfig/HCP_S1200_GroupAvg_v1/S1200.' + hemi + '.midthickness_MSMAll.32k_fs_LR.surf.gii '+ surf_prefix + hemi + '.shape.gii -enclosing'
    os.system(cmd)
    cmd = './wb_command -metric-dilate ' + surf_prefix + hemi + '.shape.gii' + ' /Users/emilyolafson/GIT/ENIGMA/testfig/HCP_S1200_GroupAvg_v1/S1200.' + hemi + '.midthickness_MSMAll.32k_fs_LR.surf.gii 20 ' + surf_prefix + hemi + '_filled.shape.gii -nearest'
    os.system(cmd)
    
